In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.size']=14
plt.rcParams['axes.grid']=True

In [ ]:
train_data_dir =    "/kaggle/input/plant-seedlings-classification/train"

In [ ]:
image_path = train_data_dir+'/Maize/006196e1c.png'
print(plt.imread(image_path).shape)
plt.imshow(plt.imread(image_path))
plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_image_generator = ImageDataGenerator(rescale=1./255,rotation_range=30, width_shift_range=0.1, 
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest",validation_split=0.25
                                           )

# val_image_generator =   ImageDataGenerator(rescale=1./255)
img_height, img_width = 200, 200
batch_size = 64

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size = batch_size,
                                                           directory  = train_data_dir,
                                                           subset='training',
                                                           shuffle    = True,
                                                           target_size = (img_height, img_width),
                                                           class_mode  ='categorical',
                                                           classes     = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet'])

In [ ]:
val_data_gen = train_image_generator.flow_from_directory(batch_size = batch_size,
                                                        directory  = train_data_dir,
                                                        subset='validation',
                                                        target_size = (img_height, img_width),
                                                        class_mode  ='categorical',
                                                        classes     = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

In [ ]:
   es = EarlyStopping(monitor = 'val_accuracy', patience = 3)
   ms = ModelCheckpoint('plant_classification', monitor = 'val_accuracy', save_best_only=True)
   model = Sequential()
   model.add(Conv2D(filters =32, kernel_size =(3,3), padding="same", input_shape=(200, 200, 3),activation="relu"))
   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
   model.add(Conv2D(filters =64,kernel_size = (3,3), padding="same",activation="relu")) 
   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
   model.add(Conv2D(filters =128, kernel_size =(3,3), padding="same",activation="relu"))   
   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
   model.add(Flatten())
  #  model.add(Dropout(0.4))
   model.add(Dense(500,activation="relu"))
   model.add(Dense(12,activation="softmax")) 
   model.compile(optimizer= 'adam',loss="categorical_crossentropy", metrics=["accuracy"])
   model.summary()

In [ ]:
%%time
history = model.fit(train_data_gen,
          steps_per_epoch= 3568//batch_size,
          epochs= 70,
          validation_data= val_data_gen,
          validation_steps= 1182//batch_size )

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label = 'train accuracy')
plt.plot(history.history['val_accuracy'], label = 'test accuracy')
plt.legend()
plt.grid(True)
plt.show()